**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning -- BARRIER Antoine

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self, e):
        self.epsilon = e

    def act(self, s, train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self, s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

act is a function which chooses the next action of the agent. As we have to learn the best action associated to each state, we need: 
1. **exploration**: the function should select all actions in order to test them and learn which one is the best, 
2. **exploitation**: the function should select more the action which gives best rewards.

To do both exploration and exploitation, we use here the $\epsilon$-greedy policy: with probability $\epsilon$ we choose a random action (**exploration**) while with probability $1-\epsilon$ we choose the best curent action (**exploitation**). 

Of course the choice of $\epsilon$ is crucial: for exploration we need a big $\epsilon$ whereas for exploitation we want a small $\epsilon$. Thus choosing a constant $\epsilon$ might not be a good strategy: at the beginning we need a big $\epsilon$ to do a lot of explorations and quickly obtain a good estimation of the best action (**training**), then we decrease $\epsilon$ to exploit the exploration and obtain big rewards. 

As we can see, the environment allows us to change the value of $\epsilon$ (with the set-epsilon function). Another way to deal with the exploration-exploitation dilemma is to set a fixed $\epsilon$ for all times but choose times for which we impose exploitation: this is the goal of the `train` parameter of the function. One strategy could be to fix a time $T$ after which we only do exploitation. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose - reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size, grid_size))
        self.position = np.zeros((grid_size, grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self, e):
        #skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
        
        fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
        fps = 30
        out = cv2.VideoWriter(str(e) + '.avi', fourcc, fps, (self.grid_size*self.scale, self.grid_size*self.scale))

        # new frame after each addition of water
        for t in range(self.max_time+2):
            frame_t = self.to_draw[t]
            frame_t = cv2.normalize(frame_t, None, 255, 0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            out.write(frame_t)
        # close out the video writer
        out.release()

    def get_frame(self, t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.board>0, 0] = 256
        b[self.board<0, 2] = 256
        b[self.x, self.y, :] = 256
        b[-2:, :, :] = 0
        b[:, -2:, :] = 0
        b[:2, :, :] = 0
        b[:, :2, :] = 0
        
        b = cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t, :, :, :] = b

    def act(self, action):
        """This function returns the new state, reward and decides if the game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size, 1),
                        self.position.reshape(self.grid_size, self.grid_size, 1)), axis=2)
        state = state[self.x-2:self.x+3, self.y-2:self.y+3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        
        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x, self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size, 1),
                        self.position.reshape(self.grid_size, self.grid_size, 1)), axis=2)

        state = state[self.x-2:self.x+3, self.y-2:self.y+3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 50 # set small when debugging
epochs_test = 50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/avi" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

As the rat has a visibility of $2$ cells, we add two rows and two columns at both the beginning and the end of the grid. Then:
- ```position``` stores the position of the rat at one time (coded with a $1$ at the corresponding cell). Note that at time $t$, after the action has been selected and the rat has moved, the position stored is the position before moving (thus at the reset no position is stored, and then after the first action the initial position is stored). 
- ```board``` stores the values of the rewards on the grid: $+0,5$ or $-1$ on the charged cells, $0$ otherwise. A cell contains a positive reward with probability ```temperature```, and the remaining cells contain a negative one with same probability. All cells which are not in the board (two first/last columns/rows) are set to $-1$ to avoid the rat to go there. 

The actions are the following: $0$ for down, $1$ for up, $2$ for right and $3$ for left. Note that if the rat is already at the frontier of the grid and decide to cross it with corresponding action, the environment forces the rat to go in the opposite action. 

The environment uses both ```position``` and ```board``` to return the current state: a $5\times 5 \times 2$ array: the first $5\times 5$ array contains the current ```board``` values of the neighboorhod of the rat, and the second one stores the last position of the rat. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        state = env.reset()    # at each epoch we reinitialize the game
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})".format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

**Note**: I had issues with the save of videos in .mp4 with ```skvideo.io.vwrite```, so I changed the ```env.draw``` function and saved .avi files. 

If you want to use again .mp4 videos, just uncomment the line below and in the ```env.draw``` function, and erase all the code associated to the .avi save. 

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix='random')
#HTML(display_videos('random0.mp4'))
HTML(display_videos('random0.avi'))

Win/lose count 10.0/16.0. Average score (-6.0)
Win/lose count 11.0/17.0. Average score (-6.0)
Win/lose count 17.0/7.0. Average score (-0.6666666666666666)
Win/lose count 11.0/19.0. Average score (-2.5)
Win/lose count 3.5/6.0. Average score (-2.5)
Win/lose count 10.0/15.0. Average score (-2.9166666666666665)
Win/lose count 11.0/14.0. Average score (-2.9285714285714284)
Win/lose count 11.5/14.0. Average score (-2.875)
Win/lose count 9.5/12.0. Average score (-2.8333333333333335)
Win/lose count 9.5/8.0. Average score (-2.4)
Win/lose count 10.5/8.0. Average score (-1.9545454545454546)
Win/lose count 11.0/19.0. Average score (-2.4583333333333335)
Win/lose count 9.0/15.0. Average score (-2.730769230769231)
Win/lose count 9.0/27.0. Average score (-3.8214285714285716)
Win/lose count 6.5/5.0. Average score (-3.466666666666667)
Win/lose count 12.5/11.0. Average score (-3.15625)
Win/lose count 10.0/6.0. Average score (-2.735294117647059)
Win/lose count 9.0/12.0. Average score (-2.75)
Win/lose coun

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




1. We have (the third line is obtained by Markov property): 
    \begin{align*}
    Q^{\pi} (s, a) &= E_{p^{\pi}} \big[\sum_{t\geq 0}\gamma^{t} r(s_{t}, a_{t}) | s_{0}=s, a_{0}=a \big] \\
        &= E[r(s, a)] + \gamma E_{p^{\pi}} \big[\sum_{t\geq 1}\gamma^{t-1} r(s_{t}, a_{t}) | s_{0}=s, a_{0}=a \big] \\
        &= E[r(s, a)] + \gamma \sum_{(s', a')} p(s' | s_0, a_0) \pi(a' | s') E_{p^{\pi}} \big[\sum_{t\geq 1}\gamma^{t-1} r(s_{t}, a_{t}) | s_{1}=s', a_{1}=a' \big] \\
        &= E[r(s, a)] + \gamma \sum_{(s', a')} p(s' | s_0, a_0) \pi(a' | s') Q^\pi(s',  a') \\
        &= E_{(s', a') \sim p^\pi(. | s, a)} [r(s, a) + \gamma Q^\pi(s', a')]
    \end{align*}

2. Then $Q^* = Q^{\pi^*}$ satisfies this last equality thus: 
   \begin{align*}
    Q^*(s, a) &= E_{(s', a') \sim p^{\pi^*}(. | s, a)} [r(s, a) + \gamma Q^*(s', a')] \\
        &= E_{s' \sim p(. | s, a)} \big[r(s, a) + \gamma E_{a' \sim \pi^*(. | s')} [Q^*(s', a')]\big] \\
        &\leq E_{s' \sim p(.|s, a)} [r(s, a) + \gamma \max_{a'} Q^*(s', a')]
    \end{align*}

    We want to show the equality in the last line. Let $\pi$ be a policy such that $a \notin \text{argmax}_{a'} Q^*(s, a')$ implies $\pi(a | s) = 0$ (in other words we select the action with maximal optimal $Q$-function). Note that $E_{a \sim \pi} [Q^*(s, a)] = \max_a Q^* (s, a)$. Then for all $(s_0, a_0)$: 

    \begin{align*}
    Q^*(s_0, a_0) &= E_{(s_1, a_1) \sim p^{\pi^*}(. | s_0, a_0)} [r(s_0, a_0) + \gamma Q^*(s_1, a_1)] \\
        &\leq E_{s_1 \sim p(. | s_0, a_0)} [r(s_0, a_0) + \gamma \max_{a_1} Q^*(s_1, a_1)] \\
        &= E_{(s_1, a_1) \sim p^\pi(. | s_0, a_0)} [r(s_0, a_0) + \gamma Q^*(s_1, a_1)] \\
        &\leq E_{(s_1, a_1) \sim p^\pi(. | s_0, a_0)} [r(s_0, a_0) + \gamma E_{(s_2, a_2) \sim p^{\pi^*}(. | s_1, a_1)} [r(s_1, a_1) + \gamma Q^*(s_2, a_2)]] \\
        &\leq E_{(s_1, a_1) \sim p^\pi(. | s_0, a_0)} [r(s_0, a_0) + \gamma E_{s_2 \sim p(. | s_1, a_1)} [r(s_1, a_1) + \gamma \max_{a_2} Q^\pi(s_2, a_2)]] \\
        &= E_{(s_1, a_1) \sim p^\pi(. | s_0, a_0)} [r(s_0, a_0) + \gamma E_{(s_2, a_2) \sim p^\pi(. | s_1, a_1)} [r(s_1, a_1) + \gamma Q^*(s_2, a_2)]] \\
        &\leq \dots \\
        &\leq E_{p^\pi} \big[\sum_{t \geq 0} \gamma^t r(s_t, a_t)\big] \\
        &= Q^\pi (s_0, a_0)
    \end{align*}

    But by definition $Q^\pi \leq Q^*$ thus we have $Q^* = Q^\pi$ and $\pi$ is an optimal policy. Then:
    $$ Q^*(s, a) = E_{(s', a') \sim p^\pi(. | s, a)} [r(s, a) + \gamma Q^*(s', a')] = E_{s' \sim p(.|s, a)} [r(s, a) + \gamma \max_{a'} Q^*(s', a')] $$

3.  Let $Q$ be a $Q$-function parameterized by $\theta$. In order to find $Q^*$, we want
    $$ \delta(s, a, \theta) = r(s, a) + \gamma \max_{a'} Q(s', a', \theta) - Q(s, a, \theta)$$
    (where $s' \sim p(. | s, a)$) to be close to $0$ in expectation (note that we can prove that $Q^*$ is the unique function satisfying $E[\delta] = 0$). Using the square loss function the objective can thus be the minimization of: 
    \begin{align*}
    \mathcal{L}(\theta) &= E \Vert (\delta(s, a, \theta))_{s, a} \Vert^2 \\
        &= E_{s' \sim p(. | s, a)} \Vert r(s, a) + \gamma \max_{a'} Q(s', a', \theta) - Q(s, a, \theta) \Vert^{2}
    \end{align*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:    # if the memory is full, we delete the first memory 
            self.memory = self.memory[1:]

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e == 0 or (e+1) % 10 == 0:
            env.draw(prefix + str(e+1))

        # Update stats
        score += win - lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5', name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size, epsilon=0.1, memory_size=100, batch_size=16, n_state=2):
        super(DQN, self).__init__(epsilon=epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of states
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None]))    # we return the action with best Q-function

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s, ns, a, r, game_over = self.memory.random_access()
            input_states[i] = s
            target_q[i] = self.model.predict(s[None])[0]
            
            if game_over_:
                target_q[i, a] = r
            else:
                target_q[i, a] = r + self.discount*np.max(self.model.predict(ns[None]))
                
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self, name_weights='model.h5', name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self, name_weights='model.h5', name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
    
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1, **kwargs):
        super(DQN_FC, self).__init__(*args, **kwargs)
        
        model = Sequential()     # model initialization
        model.add(Flatten(input_shape=(5, 5, self.n_state,)))     # transformation to a 1D array
        model.add(Dense(32))     # one linear layer with 32 outputs
        model.add(Activation('relu'))     # activation function
        model.add(Dense(4))     # one linear layer to obtain the feature associated to each action
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon=0.1, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix='fc_train')
#HTML(display_videos('fc_train10.mp4'))
HTML(display_videos('fc_train10.avi'))

Epoch 001/050 | Loss 1.0901 | Win/lose count 2.5/4.0 (-1.5)
Epoch 002/050 | Loss 1.9918 | Win/lose count 3.0/5.0 (-2.0)
Epoch 003/050 | Loss 1.7876 | Win/lose count 3.0/4.0 (-1.0)
Epoch 004/050 | Loss 1.7541 | Win/lose count 1.5/3.0 (-1.5)
Epoch 005/050 | Loss 1.7639 | Win/lose count 3.0/3.0 (0.0)
Epoch 006/050 | Loss 1.9792 | Win/lose count 6.0/9.0 (-3.0)
Epoch 007/050 | Loss 1.9744 | Win/lose count 5.5/0 (5.5)
Epoch 008/050 | Loss 2.0072 | Win/lose count 5.0/2.0 (3.0)
Epoch 009/050 | Loss 2.0032 | Win/lose count 6.0/4.0 (2.0)
Epoch 010/050 | Loss 1.9078 | Win/lose count 5.0/2.0 (3.0)
Epoch 011/050 | Loss 1.8934 | Win/lose count 8.0/7.0 (1.0)
Epoch 012/050 | Loss 1.9609 | Win/lose count 3.0/6.0 (-3.0)
Epoch 013/050 | Loss 1.8672 | Win/lose count 7.0/6.0 (1.0)
Epoch 014/050 | Loss 1.9508 | Win/lose count 6.5/3.0 (3.5)
Epoch 015/050 | Loss 1.8838 | Win/lose count 4.5/4.0 (0.5)
Epoch 016/050 | Loss 1.8546 | Win/lose count 3.0/4.0 (-1.0)
Epoch 017/050 | Loss 1.8244 | Win/lose count 3.5/3.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [14]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(64, (3, 3), input_shape=(5, 5, self.n_state,)))     # first convolutional layer
        model.add(Activation('relu'))
        model.add(Conv2D(32, (2, 2)))     # second convolutional layer
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon=0.1, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix='cnn_train')
#HTML(display_videos('cnn_train10.mp4'))
HTML(display_videos('cnn_train10.avi'))

Epoch 001/050 | Loss 0.0143 | Win/lose count 1.5/3.0 (-1.5)
Epoch 002/050 | Loss 0.0056 | Win/lose count 3.5/5.0 (-1.5)
Epoch 003/050 | Loss 0.0077 | Win/lose count 1.0/4.0 (-3.0)
Epoch 004/050 | Loss 0.0095 | Win/lose count 4.0/2.0 (2.0)
Epoch 005/050 | Loss 0.0132 | Win/lose count 3.5/1.0 (2.5)
Epoch 006/050 | Loss 0.0245 | Win/lose count 2.0/3.0 (-1.0)
Epoch 007/050 | Loss 0.0292 | Win/lose count 3.5/1.0 (2.5)
Epoch 008/050 | Loss 0.0499 | Win/lose count 2.0/2.0 (0.0)
Epoch 009/050 | Loss 0.0683 | Win/lose count 5.0/2.0 (3.0)
Epoch 010/050 | Loss 0.1461 | Win/lose count 2.0/3.0 (-1.0)
Epoch 011/050 | Loss 0.2614 | Win/lose count 1.5/0 (1.5)
Epoch 012/050 | Loss 0.5224 | Win/lose count 7.0/4.0 (3.0)
Epoch 013/050 | Loss 1.4027 | Win/lose count 5.5/5.0 (0.5)
Epoch 014/050 | Loss 1.9594 | Win/lose count 1.0/2.0 (-1.0)
Epoch 015/050 | Loss 1.8896 | Win/lose count 3.5/4.0 (-0.5)
Epoch 016/050 | Loss 1.9445 | Win/lose count 3.0/4.0 (-1.0)
Epoch 017/050 | Loss 1.7181 | Win/lose count 7.0/3

In [16]:
HTML(display_videos('cnn_train50.avi'))

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [21]:
env = Environment(grid_size=size, max_time=T, temperature=0.9)
agent_cnn = DQN_CNN(size, lr=.1, epsilon=0.1, memory_size=2000, batch_size=32)
agent_cnn.load(name_weights='cnn_trainmodel.h5', name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon=0.1, memory_size=2000, batch_size=32)
agent_cnn.load(name_weights='fc_trainmodel.h5', name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn, env, epochs_test, prefix='cnn_test')
print('Test of the FC')
test(agent_fc, env, epochs_test, prefix='fc_test')

Test of the CNN
Win/lose count 42.5/1.0. Average score (41.5)
Win/lose count 15.5/1.0. Average score (28.0)
Win/lose count 40.0/0. Average score (32.0)
Win/lose count 40.5/2.0. Average score (33.625)
Win/lose count 44.5/1.0. Average score (35.6)
Win/lose count 62.0/2.0. Average score (39.666666666666664)
Win/lose count 26.5/1.0. Average score (37.642857142857146)
Win/lose count 12.0/0. Average score (34.4375)
Win/lose count 59.0/3.0. Average score (36.833333333333336)
Win/lose count 57.0/2.0. Average score (38.65)
Win/lose count 57.0/4.0. Average score (39.95454545454545)
Win/lose count 48.5/1.0. Average score (40.583333333333336)
Win/lose count 54.0/0. Average score (41.61538461538461)
Win/lose count 56.5/3.0. Average score (42.464285714285715)
Win/lose count 49.0/2.0. Average score (42.766666666666666)
Win/lose count 39.0/2.0. Average score (42.40625)
Win/lose count 48.5/2.0. Average score (42.64705882352941)
Win/lose count 45.0/0. Average score (42.77777777777778)
Win/lose count 64.

In [18]:
#HTML(display_videos('cnn_test10.mp4'))
HTML(display_videos('cnn_test10.avi'))

In [19]:
#HTML(display_videos('fc_test10.mp4'))
HTML(display_videos('fc_test10.avi'))

We have to increase the temperature in order to obtain good results, that is to say increase the number of initial bonus and malus on the grid. We obtain a really good result with a temperature of $0,9$. In all situations, the main problem we are facing is a lack of exploration: many times we observe oscillations of two states and thus no discovery of new positions of the grid. This is particularly the case with the FC agent, but we can also observe it sometimes with the CNN agent. Yet this last agent is the best with same number of training epochs. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.

I did not implement it. I apologize for this. 

In [ ]:
def train_explore(agent, env, epoch, prefix=''):
    pass
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        pass
    
## use those samples of code:
#In train explore:
state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
reward = 0
if train:
    reward = -self.malus_position[self.x, self.y]
self.malus_position[self.x, self.y] = 0.1

reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1), 
                                self.board.reshape(self.grid_size, self.grid_size,1), 
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [ ]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon=0.1, memory_size=2000, batch_size=32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

In [ ]:
# Evaluation
test(agent, env, epochs_test, prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***